In [1]:
import cv2
import numpy as np

In [ ]:
def read_images(img_dir):

    # Define the image file extensions to be read
    img_exts = (".jpg", ".jpeg", ".png")

    # Initialize an empty list to store the images
    images = []

    # Loop through all the files in the directory
    for file in sorted(os.listdir(img_dir)):
        # Check if the file has a valid image extension
        if file.lower().endswith(img_exts):
            # Read the image using OpenCV
            img = cv2.imread(os.path.join(img_dir, file))
            # # Convert the image to grayscale and append it to the list
            # gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            images.append(img)
            

    # Convert the list of images to a 2D numpy array
    img_array = np.array(images)

    return img_array

In [2]:
def feature_descriptor(img, corners, patch_size=16, num_sub_blocks=4, num_bins=8):
    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Compute the gradient magnitudes and orientations of the entire image
    dx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    dy = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    mag = np.sqrt(dx**2 + dy**2)
    ang = np.arctan2(dy, dx) * 180 / np.pi

    # Initialize an empty array to store the feature descriptors
    descriptors = np.zeros((len(corners), num_sub_blocks**2 * num_bins))

    # Loop over each detected corner
    for i, corner in enumerate(corners):
        x, y = corner

        # Extract a patch of pixels centered around the corner
        half_patch_size = patch_size // 2
        patch_mag = mag[x-half_patch_size:x+half_patch_size+1, y-half_patch_size:y+half_patch_size+1]
        patch_ang = ang[x-half_patch_size:x+half_patch_size+1, y-half_patch_size:y+half_patch_size+1]

        # Divide the patch into sub-blocks and compute histograms of gradient orientations
        sub_block_size = patch_size // num_sub_blocks
        descriptor = np.zeros(num_sub_blocks**2 * num_bins)
        bin_width = 360 / num_bins
        for j in range(num_sub_blocks):
            for k in range(num_sub_blocks):
                sub_patch_mag = patch_mag[j*sub_block_size:(j+1)*sub_block_size, k*sub_block_size:(k+1)*sub_block_size]
                sub_patch_ang = patch_ang[j*sub_block_size:(j+1)*sub_block_size, k*sub_block_size:(k+1)*sub_block_size]
                hist, _ = np.histogram(sub_patch_ang, bins=num_bins, range=(0, 360), weights=sub_patch_mag)
                descriptor[j*num_sub_blocks*num_bins + k*num_bins:j*num_sub_blocks*num_bins + (k+1)*num_bins] = hist

        # Normalize the descriptor to make it invariant to changes in illumination and contrast
        descriptor /= np.linalg.norm(descriptor)

        # Add the descriptor to the list of descriptors
        descriptors[i] = descriptor

    return descriptors


In [ ]:
# Draw the descriptors on the image
sift_img = img.copy()
for i, corner in enumerate(corners):
    x, y = corner
    descriptor = descriptors[i]
    for j in range(len(descriptor)):
        value = int(descriptor[j] * 255)
        color = (value, value, value)
        x1 = x + j % 16 - 8
        y1 = y + j // 16 - 8
        x2 = x1 + 1
        y2 = y1 + 1
        cv2.rectangle(sift_img, (x1, y1), (x2, y2), color, -1)

# Show the image
cv2.imshow('SIFT descriptors', sift_img)
cv2.waitKey(0)
cv2.destroyAllWindows()